In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install librosa soundfile numpy sklearn pyaudio

  ERROR: Failed building wheel for pyaudio
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-xaqtf9qn/pyaudio_75344c9f51ef4674a1ae22181fedf18a/setup.py'"'"'; __file__='"'"'/tmp/pip-install-xaqtf9qn/pyaudio_75344c9f51ef4674a1ae22181fedf18a/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-66n3j0c8/install-record.txt --single-version-externally-managed --compile --install-headers /usr/local/include/python3.7/pyaudio Check the logs for full command output.


In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
  with soundfile.SoundFile(file_name) as sound_file:
    X = sound_file.read(dtype="float32")
    sample_rate=sound_file.samplerate
    if chroma:
      stft=np.abs(librosa.stft(X))
      result=np.array([])
    if mfcc:
      mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
      result=np.hstack((result, mfccs))
    if chroma:
      chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
      result=np.hstack((result, chroma))
    if mel:
      mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
      result=np.hstack((result, mel))
  return result

In [ ]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['sad', 'happy','neutral','angry']

In [ ]:
import os
output_file_path = "drive/MyDrive/FrostHack/data"
files = os.listdir(output_file_path)
def load_data(test_size=0.2):
  x,y=[],[]
  for file in files:
    if(file!="x_train.npy" and file!="y_train.npy" and file!="x_test.npy" and file!="y_test.npy"):
      output_file_path = 'drive/MyDrive/FrostHack/data/'+file +"/"
      for video in os.listdir(output_file_path):
        emotion=emotions[video.split("-")[2]]
        if(emotion in observed_emotions):
          feature=extract_feature('drive/MyDrive/FrostHack/data/'+file +"/"+video, mfcc=True, chroma=True, mel=True)
          x.append(feature)
          y.append(emotion)
  return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
# x_train = np.load("drive/MyDrive/FrostHack/data/x_train.npy")
# y_train = np.load("drive/MyDrive/FrostHack/data/y_train.npy")
# x_test = np.load("drive/MyDrive/FrostHack/data/x_test.npy")
# y_test = np.load("drive/MyDrive/FrostHack/data/y_test.npy")

In [ ]:
path = 'drive/MyDrive/FrostHack/data/y_test'
np.save(path,y_test)

In [ ]:
# mlp_gs = MLPClassifier(max_iter=500)
# parameter_space = {
#     'hidden_layer_sizes': [(100,200,100),(200,300,200)],
#     'activation': ['tanh', 'relu'],
#     'alpha': [0.0001,0.01, 0.05,0.1],
#     'learning_rate': ['constant','adaptive'],
#     'batch_size' : [128,256,32],
# }
# from sklearn.model_selection import GridSearchCV
# clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
# clf.fit(x_train, y_train)

In [ ]:
# clf.best_params_

In [ ]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.000001 ,batch_size= 32,activation= 'relu',epsilon=1e-08, hidden_layer_sizes=(500, 500, 500), learning_rate='adaptive', max_iter=500)

In [ ]:
# Initialize the Multi Layer Perceptron Classifier
# model=MLPClassifier(alpha=0.0001, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(1000,), learning_rate='constant', max_iter=500)

In [ ]:
model.fit(x_train,y_train)

MLPClassifier(alpha=1e-06, batch_size=32, hidden_layer_sizes=(500, 500, 500),
              learning_rate='adaptive', max_iter=500)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['happy', 'angry', 'sad', 'happy', 'sad', 'happy', 'angry', 'sad',
       'neutral', 'sad', 'sad', 'sad', 'happy', 'neutral', 'happy',
       'angry', 'angry', 'neutral', 'neutral', 'happy', 'sad', 'angry',
       'happy', 'sad', 'sad', 'angry', 'sad', 'sad', 'happy', 'sad',
       'sad', 'angry', 'angry', 'angry', 'angry', 'sad', 'sad', 'sad',
       'neutral', 'happy', 'sad', 'sad', 'sad', 'happy', 'sad', 'angry',
       'sad', 'sad', 'neutral', 'sad', 'happy', 'sad', 'angry', 'neutral',
       'sad', 'angry', 'sad', 'sad', 'sad', 'angry', 'angry', 'angry',
       'neutral', 'happy', 'happy', 'happy', 'happy', 'angry', 'angry',
       'happy', 'angry', 'angry', 'sad', 'angry', 'sad', 'angry', 'happy',
       'angry', 'angry', 'happy', 'angry', 'angry', 'angry', 'sad', 'sad',
       'happy', 'angry', 'sad', 'happy', 'happy', 'angry', 'happy',
       'happy', 'angry', 'angry', 'neutral', 'sad', 'happy', 'happy',
       'angry', 'happy', 'sad', 'angry', 'happy', 'happy', 'neutral'

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))